Model to generate a sequence of following words:
1. Extract relevant characters data
2. Convert data to token sequence of size 5
3. Encode the sentence 
4. Create a bidirectional LSTM model
5. Add Glove word embeddings

Reference:

~ https://medium.com/@plusepsilon/the-bidirectional-language-model-1f3961d1fb27

~ https://www.kaggle.com/guidant/mimicking-star-wars-characters-using-a-i-rnn#2.-Data-Preparation

~ https://machinelearningmastery.com/how-to-develop-a-word-level-neural-language-model-in-keras/

~ https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/

~ https://towardsdatascience.com/simple-text-generation-d1c93f43f340



In [122]:
# Imports
import numpy as np
import pandas as pd
import string
from random import randint
from tensorflow.keras.preprocessing.sequence import pad_sequences

from keras.models import Sequential
from keras.models import Model
from keras.models import load_model
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import GRU
from keras.layers import Embedding
from keras.layers import Input
from keras.layers import Bidirectional
from keras.layers import GlobalMaxPool1D
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.text import Tokenizer
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping


from __future__ import print_function
from nltk.translate.bleu_score import SmoothingFunction, corpus_bleu, sentence_bleu

# Get Pre-trained GLOVE embeddings
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove*.zip
!ls
!pwd

#load the whole embedding into memory
embeddings_index = dict()
f = open('/content/glove.6B.300d.txt')

for line in f:
  values = line.split()
  word = values[0]
  coefs = np.asarray(values[1:], dtype='float32')
  embeddings_index[word] = coefs

f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

--2021-04-22 01:37:29--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2021-04-22 01:37:29--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2021-04-22 01:37:29--  http://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip.6’

glove

In [123]:
# Add file path to where the filtered data may be
# One can also add the folder temporarily in the runtime location
# path_to_file = '/content/Filtered Data/'
path_to_file = '/content/drive/MyDrive/SNLP Project/Filtered Data/'

# Create Dataframe of all characters and dialogues
data = pd.DataFrame(columns = ['character', 'dialogue'])

for file in os.listdir(path_to_file):
    print(file)
    df = pd.read_csv(path_to_file+file)
    data = data.append(df, ignore_index=True)

data['character'] = data["character"].str.lower()

data['character'] = data.character.replace("obi-wan", "ben", regex=True)
data['character'] = data.character.replace("c-3po", "threepio", regex=True)

unique_characters = data.character.unique()

data_dict = data.groupby('character')['dialogue'].apply(lambda g: g.values.tolist()).to_dict()

SW_EpisodeI.csv
SW_EpisodeII.csv
SW_EpisodeIII.csv
SW_EpisodeIV.csv
SW_EpisodeV.csv
SW_EpisodeVI.csv


In [124]:
def preprocess_text(sen):

    # Remove ....
    sentence = re.sub('\.+', ' ', sen)

    # Remove punctuations
    sentence = re.sub('[%s]' % re.escape(string.punctuation), '', sentence)

    # Remove extra spaces
    sentence = re.sub(' +', ' ', sentence)

    # Remove numbers
    sentence = ''.join(filter(lambda x: not x.isdigit(), sentence))

    # Lower case
    sentence = sentence.lower()

    # Return a list of tokens (words)
    sentence = sentence.split()

    return sentence

In [125]:
def get_char_df(name, preprocessed_sen_filename):

  global tokens, char_data, token_list

  tokens = [preprocess_text(row) for row in data_dict[name]]
  char_data = [' '.join(row) for row in tokens]
  token_list = [item for sublist in tokens for item in sublist]

  preprocessed_output_file = open(preprocessed_sen_filename,"a")
  for row in tokens:
    str_val = ' '.join(row)
    preprocessed_output_file.writelines(str_val + '\n')
  preprocessed_output_file.close() 

In [126]:
def model_gen(name):

  global tokenizer, reverse_word_dict, dataX

  # integer encode sequences of words
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(token_list)
  unique_words = set(token_list)
  sequences_tokenised = tokenizer.texts_to_sequences(tokens)

  vocab_size = len(unique_words)+1
  n_sentences = len(tokens)

  print("# of Unique Words: ", unique_words)
  print("Tokenised sequences ", sequences_tokenised)
  print("Vocabulary Size: ", vocab_size)
  print("Number of sentences ", n_sentences)

  reverse_word_dict = {v: k for k, v in tokenizer.word_index.items()} 

  # create a weight matrix for words in training docs
  embedding_matrix = np.zeros((vocab_size, 300))

  for word, i in tokenizer.word_index.items():
      embedding_vector = embeddings_index.get(word)
      if embedding_vector is not None:
          embedding_matrix[i] = embedding_vector

  #Create sliding windows of size 5
  seq_length = 5
  dataX = []
  dataY = []
  for dialogue in sequences_tokenised:
    for i in range(len(dialogue)-5):
      dataX.append(dialogue[i:i+5])
      dataY.append(dialogue[i+5])

  y = np.zeros((len(dataY), vocab_size), dtype=np.bool)
  for i, sentence in enumerate(dataY):
    y[i, dataY[i]] = 1

  X = pad_sequences(dataX, maxlen=5)

  learning_rate = 0.001
  embedding_layer = Embedding(vocab_size,
                                300,
                                weights=[embedding_matrix],
                                input_length=seq_length,
                                trainable=False)
  inp = Input(shape=(seq_length,))
  x = embedding_layer(inp)
  x = Bidirectional(LSTM(200,return_sequences=True,dropout=0.1,recurrent_dropout=0.1))(x)
  x = GlobalMaxPool1D()(x)
  x = Dense(vocab_size,activation='relu')(x)
  x = Dense(vocab_size,activation='sigmoid')(x)
  model = Model(inputs=inp,outputs=x)
  optimizer = Adam(lr=learning_rate)

  #call the functions in the metrics 
  model.compile(optimizer=optimizer,loss='categorical_crossentropy',metrics=['accuracy'])
  model.summary()

  batch_size = 32 # minibatch size
  num_epochs = 100 # number of epochs

  callbacks=[EarlyStopping(patience=4, monitor='val_loss'),
              ModelCheckpoint(filepath="./" + name + 'model_gen_sentences.{epoch:02d}-{val_loss:.2f}.hdf5',\
                              monitor='val_loss', verbose=0, mode='auto', period=2)]
  #fit the model
  history = model.fit(X, y,
                    batch_size=batch_size,
                    shuffle=True,
                    epochs=num_epochs,
                    callbacks=callbacks,
                    validation_split=0.1)

  #save the model
  model.save("/content/drive/MyDrive/Colab Notebooks/" + name + "_model_generate_sentences.h5")


In [127]:
def sample(preds, temperature=2.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [128]:
def generate_sentence(model):

  #initiate sentences
  generated = ''
  sentence = dataX[randint(0,len(dataX))]
  sentence = [' '.join([reverse_word_dict[word] for word in sentence])]
  generated += sentence[0]

  #we generate the next text
  for i in range(5):
    
      seq = tokenizer.texts_to_sequences(sentence)
      padded = pad_sequences(seq, maxlen=5)

      #calculate next word
      preds = model.predict(padded, verbose=0)[0]
      next_index = sample(preds, 0.33)
      next_word = reverse_word_dict[next_index]

      #add the next word to the text
      generated += " " + next_word
      sentence = [' '.join(sentence[0].split()[1:]) + " " + next_word]
      
  #print the whole text
  return (generated)

In [129]:
def get_bleu_score(ref, gen):
  bleu_score = 0
  for sen in gen:
    bleu_score += sentence_bleu(ref, sen.split())
  return (bleu_score/10)

In [130]:
def main(name, preprocessed_sen_filename, output_sen_filename):

  # Get individual character Dataframe
  get_char_df(name, preprocessed_sen_filename)

  # Prepare and train the model
  model_gen(name)

  # Load the trained model
  model = load_model("/content/drive/MyDrive/Colab Notebooks/" + name + "_model_generate_sentences.h5")

  # Generate 10 random sentences from 10 randomly selected seed sentences from original tokens
  n_sentences_gen = 10
  generated_sentences = []
  for i in range(n_sentences_gen):
    generated_sentences.append(generate_sentence(model))

  # Create output file for generated sentences
  output_file = open(output_sen_filename,"a")
  for row in generated_sentences:
    output_file.writelines(row + '\n')
  

  # Check Bleu Score of the output
  output_file.writelines('\n')
  output_file.writelines('BLEU score for {} -> {}'.format(name, get_bleu_score(tokens, generated_sentences)))

  output_file.close()


In [131]:
main("ben", "ObiWanProcessedData.txt","ObiWanGeneratedSen.txt")

# of Unique Words:  {'attuned', 'terrible', 'palpatine', 'affairs', 'rough', 'held', 'stressed', 'lead', 'suggestions', 'bad', 'learn', 'bounty', 'switch', 'obiwan', 'suit', 'empire', 'twenty', 'cody', 'takes', 'rim', 'makes', 'fast', 'tests', 'mentioned', 'that', 'plot', 'would', 'running', 'mlady', 'dagobah', 'not', 'youd', 'most', 'burden', 'seduced', 'padmei', 'asked', 'accurate', 'form', 'moody', 'coming', 'roomcheck', 'direction', 'field', 'whats', 'going', 'own', 'people', 'fins', 'politicians', 'opinionated', 'continuinghere', 'jedis', 'lightly', 'vader', 'got', 'many', 'mark', 'diplomatic', 'busy', 'crossed', 'personal', 'activity', 'term', 'came', 'allegiance', 'idealistic', 'from', 'maybe', 'sorry', 'given', 'deceive', 'no', 'arfour', 'clumsy', 'shut', 'her', 'thearchives', 'shall', 'file', 'hunt', 'so', 'whohired', 'haveerased', 'felt', 'council', 'ah', 'pathetic', 'get', 'rash', 'why', 'careful', 'two', 'theinterior', 'buzz', 'beengiven', 'certainly', 'evil', 'level', 'dea

Epoch 1/100
102/102 [==============================] - 16s 106ms/step - loss: 6.6891 - accuracy: 0.0344 - val_loss: 6.3146 - val_accuracy: 0.0556
Epoch 2/100
102/102 [==============================] - 10s 100ms/step - loss: 5.7692 - accuracy: 0.0674 - val_loss: 6.3780 - val_accuracy: 0.0611
Epoch 3/100
102/102 [==============================] - 10s 101ms/step - loss: 5.2562 - accuracy: 0.0855 - val_loss: 6.4696 - val_accuracy: 0.0778
Epoch 4/100
102/102 [==============================] - 10s 100ms/step - loss: 4.3021 - accuracy: 0.1337 - val_loss: 7.0104 - val_accuracy: 0.0722
Epoch 5/100
102/102 [==============================] - 10s 101ms/step - loss: 3.0655 - accuracy: 0.2794 - val_loss: 8.0225 - val_accuracy: 0.0528


In [133]:
main("yoda", "YodaProcessedData.txt","YodaGeneratedSen.txt")

# of Unique Words:  {'lets', 'ourselves', 'aboutpolitics', 'have', 'missingfrom', 'palpatine', 'luke', 'safe', 'learn', 'immortality', 'opinion', 'today', 'with', 'solitude', 'barely', 'through', 'nevertheless', 'obiwan', 'little', 'dismantling', 'influence', 'out', 'you', 'opens', 'feeling', 'rim', 'makes', 'respect', 'assassin', 'hundredjedi', 'clonewar', 'wait', 'friend', 'that', 'strong', 'kenobi', 'war', 'would', 'powers', 'must', 'obiwanyounglings', 'welcome', 'how', 'sidious', 'creates', 'kashyyyk', 'not', 'wemust', 'sky', 'or', 'our', 'most', 'burden', 'sleep', 'temple', 'doing', 'confident', 'one', 'maste', 'security', 'grievous', 'only', 'windu', 'obiwans', 'tothe', 'agree', 'tatooine', 'shadow', 'continuingthe', 'remains', 'powerful', 'droid', 'own', 'nothing', 'too', 'hard', 'chewbacca', 'then', 'transform', 'lightly', 'jedis', 'vader', 'oh', 'misread', 'just', 'failure', 'goand', 'yes', 'many', 'puzzle', 'agent', 'other', 'impossible', 'things', 'eveything', 'twilight', 't

Epoch 1/100
38/38 [==============================] - 9s 97ms/step - loss: 6.1641 - accuracy: 0.0399 - val_loss: 5.6117 - val_accuracy: 0.0746
Epoch 2/100
38/38 [==============================] - 3s 81ms/step - loss: 5.3495 - accuracy: 0.0615 - val_loss: 5.6549 - val_accuracy: 0.0896
Epoch 3/100
38/38 [==============================] - 3s 83ms/step - loss: 5.0599 - accuracy: 0.0825 - val_loss: 5.5618 - val_accuracy: 0.1269
Epoch 4/100
38/38 [==============================] - 3s 80ms/step - loss: 4.6229 - accuracy: 0.1257 - val_loss: 5.6796 - val_accuracy: 0.1642
Epoch 5/100
38/38 [==============================] - 3s 81ms/step - loss: 3.9411 - accuracy: 0.1737 - val_loss: 5.7832 - val_accuracy: 0.1343
Epoch 6/100
38/38 [==============================] - 3s 81ms/step - loss: 2.9378 - accuracy: 0.3089 - val_loss: 6.7067 - val_accuracy: 0.1642
Epoch 7/100
38/38 [==============================] - 3s 81ms/step - loss: 1.7431 - accuracy: 0.5592 - val_loss: 7.2601 - val_accuracy: 0.1418


In [135]:
main("threepio", "ThreepioProcessedData.txt","ThreepioGeneratedSen.txt")

# of Unique Words:  {'venture', 'lets', 'oil', 'rusty', 'hasnt', 'better', 'somethingshappened', 'behave', 'curse', 'ourselves', 'hangar', 'have', 'commented', 'tricked', 'luke', 'expect', 'safe', 'replace', 'readily', 'alternative', 'bad', 'deity', 'itif', 'hersoh', 'opinion', 'with', 'ever', 'needing', 'bounty', 'bear', 'barely', 'lars', 'through', 'aatwentythree', 'switch', 'obiwan', 'load', 'impersonate', 'damage', 'leia', 'said', 'least', 'entirely', 'little', 'worries', 'dull', 'saying', 'weight', 'empire', 'delusions', 'out', 'sent', 'fortunate', 'you', 'havent', 'alliance', 'veranda', 'person', 'overweight', 'twenty', 'peculiar', 'feeling', 'naked', 'wooly', 'heading', 'thirtyfive', 'makes', 'mischka', 'surrender', 'extreme', 'supposed', 'bolt', 'fast', 'wait', 'maam', 'odds', 'friend', 'therell', 'that', 'rocky', 'approximately', 'ey', 'mutualattraction', 'kenobi', 'navigating', 'shutting', 'would', 'million', 'condition', 'responsibility', 'must', 'welcome', 'quitefound', 'ta

Epoch 1/100
72/72 [==============================] - 12s 98ms/step - loss: 6.5475 - accuracy: 0.0249 - val_loss: 6.1571 - val_accuracy: 0.0431
Epoch 2/100
72/72 [==============================] - 6s 89ms/step - loss: 5.7599 - accuracy: 0.0368 - val_loss: 6.0879 - val_accuracy: 0.0588
Epoch 3/100
72/72 [==============================] - 6s 90ms/step - loss: 5.2199 - accuracy: 0.0636 - val_loss: 6.1926 - val_accuracy: 0.0980
Epoch 4/100
72/72 [==============================] - 6s 88ms/step - loss: 4.2393 - accuracy: 0.1338 - val_loss: 6.6766 - val_accuracy: 0.1059
Epoch 5/100
72/72 [==============================] - 6s 89ms/step - loss: 2.9161 - accuracy: 0.3314 - val_loss: 7.3731 - val_accuracy: 0.0863
Epoch 6/100
72/72 [==============================] - 6s 88ms/step - loss: 1.5457 - accuracy: 0.6296 - val_loss: 8.5281 - val_accuracy: 0.0706
